<a href="https://colab.research.google.com/github/avidday/Colab/blob/main/nvcc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17163259664792253899
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14328594432
locality {
  bus_id: 1
  links {
  }
}
incarnation: 17891940015618096752
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
xla_global_id: 416903419
]


In [ ]:
!ls /usr/local/cuda/
!nvcc --version

bin	compute-sanitizer  extras  include  nvml  share  targets
compat	doc		   gds	   lib64    nvvm  src
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [23]:
%%writefile p4x4det.cu
#include <iostream>
#include <vector>
#include <algorithm>
#include <cstdio>
#include <cub/block/block_load.cuh>
#include <cub/block/block_store.cuh>
#include <cub/block/block_exchange.cuh>
#include <cuda/std/complex>

#define gpuErrchk(ans) { gpuAssert((ans), __FILE__, __LINE__); }
inline void gpuAssert(cudaError_t code, const char *file, int line, bool abort=true)
{
   if (code != cudaSuccess) 
   {
      fprintf(stderr,"GPUassert: %s %s %d\n", cudaGetErrorString(code), file, line);
      if (abort) exit(code);
   }
}

const int _3x3 = 9;
const int _4x4 = 16;
const int _5x5 = 25;
const int blocksz = 32;

template<typename T, bool verbose=false>
__device__ __host__ __forceinline__ 
T det2x2(T a11, T a12, 
         T a21, T a22)
{
    auto mul = [&] (T x, T y)      { return x * y; };
    auto fma = [&] (T x, T y, T z) { return mul(x, y) + z; };

 
    T a12_a21 = mul(-a12, a21);
    T det = fma(a11, a22, a12_a21);

    if (verbose) {
        printf("%f %f\n", a11, a12);
        printf("%f %f\n", a21, a22);
        printf("det = %f\n", det);
    }

    return det;
}

template<typename T, bool verbose=false>
__device__ __host__ __forceinline__ 
T det3x3(T a11, T a12, T a13, 
         T a21, T a22, T a23, 
         T a31, T a32, T a33)
{
    auto mul = [&] (T x, T y)      { return x * y; };
    auto fma = [&] (T x, T y, T z) { return mul(x, y) + z; };

    T a23_a32 = mul(-a23, a32);
    T a13_a32 = mul(-a13, a32);
    T a13_a22 = mul(-a13, a22);

    T det_22_33 = det2x2<T,verbose>(a22, a23,
                                    a32, a33);
    T det_12_13 = det2x2<T,verbose>(a12, a13,
                                    a32, a33);
    T det_12_23 = det2x2<T,verbose>(a12, a13, 
                                    a22, a23);

    T det = mul( a11, det_22_33);
    det =   fma(-a21, det_12_13, det);
    det =   fma( a31, det_12_23, det);

    if (verbose) {
        printf("%f %f %f\n", a11, a12, a13);
        printf("%f %f %f\n", a21, a22, a23);
        printf("%f %f %f\n", a31, a32, a33);

        printf("det = %f\n", det);
    }

    return det;
}

template <typename T, bool verbose=false>
__device__ __host__ __forceinline__
T det4x4(T a11, T a12, T a13, T a14, 
         T a21, T a22, T a23, T a24, 
         T a31, T a32, T a33, T a34, 
         T a41, T a42, T a43, T a44)
{
    auto mul = [&] (T x, T y)      { return x * y; };
    auto fma = [&] (T x, T y, T z) { return mul(x, y) + z; };

    T det_22_33_44 = det3x3<T,false>
                           (a22, a23, a24,
                            a32, a33, a34,
                            a42, a43, a44);

    T det_12_33_44 = det3x3<T,false>
                           (a12, a13, a14,
                            a32, a33, a34,
                            a42, a43, a44);

    T det_12_23_44 = det3x3<T,false>
                           (a12, a13, a14,
                            a22, a23, a24,
                            a42, a43, a44);

    T det_12_23_34 = det3x3<T,false>
                           (a12, a13, a14,
                            a22, a23, a24,
                            a32, a33, a34);

    T det;
    det = mul( a11, det_22_33_44);
    det = fma(-a21, det_12_33_44, det);
    det = fma( a31, det_12_23_44, det);
    det = fma(-a41, det_12_23_34, det);

    if (verbose) {
        printf("%f %f %f %f\n", a11, a12, a13, a14);
        printf("%f %f %f %f\n", a21, a22, a23, a24);
        printf("%f %f %f %f\n", a31, a32, a33, a34);
        printf("%f %f %f %f\n", a41, a42, a43, a44);

        printf("det = %f\n", det);
    }

    return det;                        
}

template <typename T, bool verbose=false>
__device__ __host__ __forceinline__
T det5x5(T a11, T a12, T a13, T a14, T a15,
         T a21, T a22, T a23, T a24, T a25,
         T a31, T a32, T a33, T a34, T a35,
         T a41, T a42, T a43, T a44, T a45,
         T a51, T a52, T a53, T a54, T a55)     
{
    auto mul = [&] (T x, T y)      { return x * y; };
    auto fma = [&] (T x, T y, T z) { return mul(x, y) + z; };

    T det_22_33_44_55 = det4x4<T,false>
                           (a22, a23, a24, a25,
                            a32, a33, a34, a35,
                            a42, a43, a44, a45,
                            a52, a53, a54, a55);

    T det_12_33_44_55 = det4x4<T,false>
                           (a12, a13, a14, a15,
                            a32, a33, a34, a35,
                            a42, a43, a44, a45,
                            a52, a53, a54, a55);

    T det_12_23_44_55 = det4x4<T,false>
                           (a12, a13, a14, a15,
                            a22, a23, a24, a25,
                            a42, a43, a44, a45,
                            a52, a53, a54, a55);

    T det_12_23_34_55 = det4x4<T,false>
                           (a12, a13, a14, a15,
                            a22, a23, a24, a25,
                            a32, a33, a34, a35,
                            a52, a53, a54, a55);

    T det_12_23_34_45 = det4x4<T,false>
                           (a12, a13, a14, a15,
                            a22, a23, a24, a25,
                            a32, a33, a34, a35,
                            a42, a43, a44, a45);
                            

    T det;
    det = mul( a11, det_22_33_44_55);
    det = fma(-a21, det_12_33_44_55, det);
    det = fma( a31, det_12_23_44_55, det);
    det = fma(-a41, det_12_23_34_55, det);
    det = fma( a51, det_12_23_34_45, det);

    if (verbose) {
        printf("%f %f %f %f %f\n", a11, a12, a13, a14, a15);
        printf("%f %f %f %f %f\n", a21, a22, a23, a24, a25);
        printf("%f %f %f %f %f\n", a31, a32, a33, a34, a35);
        printf("%f %f %f %f %f\n", a41, a42, a43, a44, a45);
        printf("%f %f %f %f %f\n", a41, a42, a43, a44, a55);

        printf("det = %f\n", det);
    }

    return det;                        
}

template <typename T, int mdim>
__global__ void detkernel(T *d_data, T *d_result, int Nmatrix)
{
    typedef cub::BlockLoad<T, blocksz, mdim, cub::BLOCK_LOAD_STRIPED> BlockLoad;
    typedef cub::BlockStore<T, blocksz, 1, cub::BLOCK_STORE_DIRECT> BlockStore;
    typedef cub::BlockExchange<T, blocksz, mdim> BlockExchange;

    __shared__ 
    union {
	    typename BlockLoad::TempStorage load;
      typename BlockStore::TempStorage store;
      typename BlockExchange::TempStorage exchange;
    } temp;

    T thread_data[mdim], l_result[1];
    T det;

    BlockLoad(temp.load).Load(d_data, thread_data, Nmatrix * mdim);
    __syncthreads();

    BlockExchange(temp.exchange).StripedToBlocked(thread_data);

    switch(mdim) {
      case _3x3:
      {
        T a11 = thread_data[0];      T a21 = thread_data[3+0];
        T a12 = thread_data[1];      T a22 = thread_data[3+1];
        T a13 = thread_data[2];      T a23 = thread_data[3+2];

        T a31 = thread_data[6+0];
        T a32 = thread_data[6+1];
        T a33 = thread_data[6+2];

        det = det3x3<T, false>( 
                        a11, a12, a13,
                        a21, a22, a23,
                        a31, a32, a33);
      } 
      break;

      case _4x4:
      {
        T a11 = thread_data[0];      T a21 = thread_data[4+0];
        T a12 = thread_data[1];      T a22 = thread_data[4+1];
        T a13 = thread_data[2];      T a23 = thread_data[4+2];
        T a14 = thread_data[3];      T a24 = thread_data[4+3];

        T a31 = thread_data[8+0];    T a41 = thread_data[12+0];
        T a32 = thread_data[8+1];    T a42 = thread_data[12+1];
        T a33 = thread_data[8+2];    T a43 = thread_data[12+2];
        T a34 = thread_data[8+3];    T a44 = thread_data[12+3];

        det = det4x4<T, false>( 
                        a11, a12, a13, a14,
                        a21, a22, a23, a24,
                        a31, a32, a33, a34,
                        a41, a42, a43, a44);
      } 
      break;

      case _5x5:
      {
        T a11 = thread_data[0];      T a21 = thread_data[5+0];
        T a12 = thread_data[1];      T a22 = thread_data[5+1];
        T a13 = thread_data[2];      T a23 = thread_data[5+2];
        T a14 = thread_data[3];      T a24 = thread_data[5+3];
        T a15 = thread_data[4];      T a25 = thread_data[5+4];

        T a31 = thread_data[10+0];    T a41 = thread_data[15+0];
        T a32 = thread_data[10+1];    T a42 = thread_data[15+1];
        T a33 = thread_data[10+2];    T a43 = thread_data[15+2];
        T a34 = thread_data[10+3];    T a44 = thread_data[15+3];
        T a35 = thread_data[10+4];    T a45 = thread_data[15+4];

        T a51 = thread_data[20+0];
        T a52 = thread_data[20+1];
        T a53 = thread_data[20+2];
        T a54 = thread_data[20+3];
        T a55 = thread_data[20+4];
        det = det5x5<T, false>( 
                        a11, a12, a13, a14, a15,
                        a21, a22, a23, a24, a25, 
                        a31, a32, a33, a34, a35, 
                        a41, a42, a43, a44, a45,
                        a51, a52, a53, a54, a55);
      }

      break;
    }

    l_result[0] = det;
    BlockStore(temp.store).Store(d_result, l_result, Nmatrix); 

}

typedef cuda::std::complex<float> cfloat;
typedef cuda::std::complex<double> cdouble;

template __global__ void detkernel<float,_3x3>(float *, float *, int);
template __global__ void detkernel<float,_4x4>(float *, float *, int);
template __global__ void detkernel<float,_5x5>(float *, float *, int);

template __global__ void detkernel<cfloat,_3x3>(cfloat *, cfloat *, int);
template __global__ void detkernel<cfloat,_4x4>(cfloat *, cfloat *, int);
template __global__ void detkernel<cfloat,_5x5>(cfloat *, cfloat *, int);

int main()
{
    const int nmatrices = 32;

    // triadiagonal laplacian 5x5
    std::vector<float> iarray(_5x5);
    iarray[0] = -4.f; iarray[1] =  2.f; iarray[2] =   0.f; iarray[3] =   0.f; iarray[4] =  0.f; 
    iarray[5] =  1.f; iarray[6] = -4.f; iarray[7] =   1.f; iarray[8] =   0.f; iarray[9] =  0.f;
    iarray[10] = 0.f; iarray[11] = 1.f; iarray[12] = -4.f; iarray[13] =  1.f; iarray[14] =  0.f;
    iarray[15] = 0.f; iarray[16] = 0.f; iarray[17] =  1.f; iarray[18] = -4.f; iarray[19] =  1.f;
    iarray[20] = 0.f; iarray[21] = 0.f; iarray[22] =  0.f; iarray[23] =  2.f; iarray[24] = -4.f;

    float det4 = det4x4<float,true>
                              (iarray[   0], iarray[   1], iarray[   2], iarray[   3],
                               iarray[ 5+0], iarray[ 5+1], iarray[ 5+2], iarray[ 5+3],
                               iarray[10+0], iarray[10+1], iarray[10+2], iarray[10+3],
                               iarray[15+0], iarray[15+1], iarray[15+2], iarray[15+3]);

    std::cout << "Reference solution 4x4 = " << det4 << std::endl;

    float det5 = det5x5<float,true>
                              (iarray[   0], iarray[   1], iarray[   2], iarray[   3], iarray[   4],
                               iarray[ 5+0], iarray[ 5+1], iarray[ 5+2], iarray[ 5+3], iarray[ 5+4],
                               iarray[10+0], iarray[10+1], iarray[10+2], iarray[10+3], iarray[10+4],
                               iarray[15+0], iarray[15+1], iarray[15+2], iarray[15+3], iarray[15+4],
                               iarray[20+0], iarray[20+1], iarray[20+2], iarray[20+3], iarray[20+4]
                               );

    std::cout << "Reference solution 5x5 = " << det5 << std::endl;

    float *d_array, *d_result;
    gpuErrchk( cudaMalloc(&d_array, nmatrices * _5x5 * sizeof(float)) );
    gpuErrchk( cudaMalloc(&d_result, nmatrices * sizeof(float)) )
    
    float *p = d_array;
    for(int i=0; i<nmatrices; i++) {
    	gpuErrchk( cudaMemcpy(p, &iarray[0], _5x5 * sizeof(float), cudaMemcpyHostToDevice) );
      p += _5x5;
      std::for_each(iarray.begin(), iarray.end(), [](float &x){ x*=1.005f;} );
    }
    
    int nblocks = (nmatrices / blocksz) + ((nmatrices % blocksz) ? 0 : 1);
    detkernel<float,_5x5><<<nblocks,blocksz>>>(d_array, d_result, nmatrices);
    gpuErrchk( cudaPeekAtLastError() );

    std::vector<float> result(nmatrices, -1.f);
    gpuErrchk( cudaMemcpy(&result[0], d_result, nmatrices * sizeof(float), cudaMemcpyDeviceToHost) );
    for(int i=0; i<nmatrices; i++) {
    	std::cout << result[i] << std::endl;
    }

    return 0;
}

Overwriting p4x4det.cu


In [28]:
!nvcc -arch=sm_75 -Xptxas="-v" -o p4x4det p4x4det.cu

p4x4det.cu(38): warning #1290-D: a class type that is not trivially copyable passed through ellipsis
          detected during:
            instantiation of "T det2x2(T, T, T, T) [with T=cfloat, verbose=false]" 
(60): here
            instantiation of "T det3x3(T, T, T, T, T, T, T, T, T) [with T=cfloat, verbose=false]" 
(227): here
            instantiation of "void detkernel<T,mdim>(T *, T *, int) [with T=cfloat, mdim=9]" 
(293): here

p4x4det.cu(38): warning #1290-D: a class type that is not trivially copyable passed through ellipsis
          detected during:
            instantiation of "T det2x2(T, T, T, T) [with T=cfloat, verbose=false]" 
(60): here
            instantiation of "T det3x3(T, T, T, T, T, T, T, T, T) [with T=cfloat, verbose=false]" 
(227): here
            instantiation of "void detkernel<T,mdim>(T *, T *, int) [with T=cfloat, mdim=9]" 
(293): here

p4x4det.cu(38): warning #181-D: argument is incompatible with corresponding format string conversion
          detecte

In [ ]:
!nvcc -arch=sm_75 -ptx p4x4det.cu
!cat p4x4det.ptx

//
// Generated by NVIDIA NVVM Compiler
//
// Compiler Build ID: CL-31833905
// Cuda compilation tools, release 11.8, V11.8.89
// Based on NVVM 7.0.1
//

.version 7.8
.target sm_75
.address_size 64

	// .globl	_Z12det5x5kernelIfEvPT_S1_i
// _ZZ12det5x5kernelIfEvPT_S1_iE4temp has been demoted
// _ZZ12det4x4kernelIfEvPT_S1_iE4temp has been demoted

.visible .entry _Z12det5x5kernelIfEvPT_S1_i(
	.param .u64 _Z12det5x5kernelIfEvPT_S1_i_param_0,
	.param .u64 _Z12det5x5kernelIfEvPT_S1_i_param_1,
	.param .u32 _Z12det5x5kernelIfEvPT_S1_i_param_2
)
{
	.reg .pred 	%p<27>;
	.reg .f32 	%f<233>;
	.reg .b32 	%r<33>;
	.reg .b64 	%rd<9>;
	// demoted variable
	.shared .align 16 .b8 _ZZ12det5x5kernelIfEvPT_S1_iE4temp[3200];

	ld.param.u64 	%rd3, [_Z12det5x5kernelIfEvPT_S1_i_param_0];
	ld.param.u64 	%rd2, [_Z12det5x5kernelIfEvPT_S1_i_param_1];
	ld.param.u32 	%r3, [_Z12det5x5kernelIfEvPT_S1_i_param_2];
	cvta.to.global.u64 	%rd4, %rd3;
	mul.lo.s32 	%r1, %r3, 25;
	mov.u32 	%r2, %tid.x;
	setp.ge.s32 	%p1, %r2

In [29]:
import numpy as np
iarray = np.empty(25)
iarray[0] = -4.;  iarray[1] =  2.; iarray[2] =   0.; iarray[3] =   0.; iarray[4] =   0.;
iarray[5] =  1.;  iarray[6] = -4.; iarray[7] =   1.; iarray[8] =   0.; iarray[9] =   0.;
iarray[10] = 0.;  iarray[11] = 1.; iarray[12] = -4.; iarray[13] =  1.; iarray[14] =  0.;
iarray[15] = 0.;  iarray[16] = 0.; iarray[17] =  1.; iarray[18] = -4.; iarray[19] =  1.;
iarray[20] = 0.;  iarray[21] = 0.;  iarray[22] = 0.; iarray[23] =  2.; iarray[24] = -4.;

x5 = np.reshape(iarray, (5,5));
print(x5)
print("5x5 result ", np.linalg.det(x5))

x = x5[:-1, :-1]
print(x)
print("4x4 result ", np.linalg.det(x))

x0 = x[:-1,:-1]
print(x0)
print("3x3 result", np.linalg.det(x0))

x1 = x[:-1,1:]
print(x1)
print("3x3 result", np.linalg.det(x1))

x2 = x[1:,:-1]
print(x2)
print("3x3 result", np.linalg.det(x2))

x3 = x[1:,1:]
print(x3)
print("3x3 result", np.linalg.det(x3))




[[-4.  2.  0.  0.  0.]
 [ 1. -4.  1.  0.  0.]
 [ 0.  1. -4.  1.  0.]
 [ 0.  0.  1. -4.  1.]
 [ 0.  0.  0.  2. -4.]]
5x5 result  -672.0
[[-4.  2.  0.  0.]
 [ 1. -4.  1.  0.]
 [ 0.  1. -4.  1.]
 [ 0.  0.  1. -4.]]
4x4 result  194.00000000000003
[[-4.  2.  0.]
 [ 1. -4.  1.]
 [ 0.  1. -4.]]
3x3 result -52.00000000000001
[[ 2.  0.  0.]
 [-4.  1.  0.]
 [ 1. -4.  1.]]
3x3 result 2.0
[[ 1. -4.  1.]
 [ 0.  1. -4.]
 [ 0.  0.  1.]]
3x3 result 1.0
[[-4.  1.  0.]
 [ 1. -4.  1.]
 [ 0.  1. -4.]]
3x3 result -56.00000000000002


In [31]:
!compute-sanitizer ./p4x4det

========= COMPUTE-SANITIZER
-4.000000 2.000000 0.000000 0.000000
1.000000 -4.000000 1.000000 0.000000
0.000000 1.000000 -4.000000 1.000000
0.000000 0.000000 1.000000 -4.000000
det = 194.000000
Reference solution 4x4 = 194
-4.000000 2.000000 0.000000 0.000000 0.000000
1.000000 -4.000000 1.000000 0.000000 0.000000
0.000000 1.000000 -4.000000 1.000000 0.000000
0.000000 0.000000 1.000000 -4.000000 1.000000
0.000000 0.000000 1.000000 -4.000000 -4.000000
det = -672.000000
Reference solution 5x5 = -672
-672
-688.969
-706.366
-724.203
-742.49
-761.238
-780.46
-800.168
-820.373
-841.088
-862.327
-884.102
-906.426
-929.315
-952.781
-976.84
-1001.51
-1026.8
-1052.72
-1079.31
-1106.56
-1134.5
-1163.15
-1192.52
-1222.63
-1253.51
-1285.16
-1317.61
-1350.88
-1384.99
-1419.96
-1455.82
========= ERROR SUMMARY: 0 errors


In [34]:
!nvprof ./p4x4det

-4.000000 2.000000 0.000000 0.000000
1.000000 -4.000000 1.000000 0.000000
0.000000 1.000000 -4.000000 1.000000
0.000000 0.000000 1.000000 -4.000000
det = 194.000000
Reference solution 4x4 = 194
-4.000000 2.000000 0.000000 0.000000 0.000000
1.000000 -4.000000 1.000000 0.000000 0.000000
0.000000 1.000000 -4.000000 1.000000 0.000000
0.000000 0.000000 1.000000 -4.000000 1.000000
0.000000 0.000000 1.000000 -4.000000 -4.000000
det = -672.000000
Reference solution 5x5 = -672
==6608== NVPROF is profiling process 6608, command: ./p4x4det
-672
-688.969
-706.366
-724.203
-742.49
-761.238
-780.46
-800.168
-820.373
-841.088
-862.327
-884.102
-906.426
-929.315
-952.781
-976.84
-1001.51
-1026.8
-1052.72
-1079.31
-1106.56
-1134.5
-1163.15
-1192.52
-1222.63
-1253.51
-1285.16
-1317.61
-1350.88
-1384.99
-1419.96
-1455.82
==6608== Profiling application: ./p4x4det
==6608== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   85.72%  44.736us